In [1]:
import torch
from det3d.torchie import Config
from det3d.models import build_detector
from det3d.torchie.trainer import load_checkpoint

config = "/workspace/centerformer/configs/nusc/nuscenes_centerformer_poolformer.py"

cfg = Config.fromfile(config)
checkpoint_path = "/workspace/centerformer/work_dirs/nuscenes_poolformer/poolformer.pth"

model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")
model.cuda()
model.eval();

/data/centerformer/det3d/core/bbox/geometry.py:149: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:162: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)
/data/centerformer/det3d/core/bbox/geometry.py:280: NumbaDeprecationWarning: The 'nopython' keyword argument was

Deformable Convolution not built!
Deformable Convolution not built!
[12/15/2023-08:20:26] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
Use HM Bias:  -2.19


In [2]:
import pickle
pickle_dir = "/workspace/centerformer/work_dirs/partition/sample_data/"

with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    x = pickle.load(handle)
        

In [3]:
with torch.no_grad():
    ct_feat, center_pos_embedding, out_scores, out_labels, out_orders, out_masks = model.neck.find_centers(x)

In [4]:
import tensorrt as trt
trt.__version__

'8.5.1.7'

In [5]:
centerFinder_engine_path = '/workspace/centerformer/work_dirs/partition/engine/findCenter_folded.trt'


def load_engine(engine_filepath):
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    with open(engine_filepath, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        engine = runtime.deserialize_cuda_engine(f.read())
    return engine

cf_engine = load_engine(centerFinder_engine_path)
cf_context = cf_engine.create_execution_context()

[12/15/2023-08:20:28] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars


In [6]:
with open(pickle_dir + "findcenter_input.pkl", 'rb') as handle:
    input_tensor = pickle.load(handle)
print((x == input_tensor).all(), x.device)

tensor(True, device='cuda:0') cuda:0


In [7]:
input_tensor.shape

torch.Size([1, 256, 180, 180])

In [8]:
to_device = torch.device("cuda")
ct_feat_tensor = torch.zeros((1, 3000, 256), dtype=torch.float32, device=to_device)
center_pos_embedding_tensor = torch.zeros((1, 3000, 256), dtype=torch.float32, device=to_device)
out_scores_tensor = torch.zeros((6, 1, 500), dtype=torch.float32, device=to_device)
out_labels_tensor = torch.zeros((6, 1, 500), dtype=torch.int32, device=to_device)
out_orders_tensor = torch.zeros((6, 1, 500), dtype=torch.int32, device=to_device)
out_masks_tensor = torch.zeros((6, 1, 500), dtype=torch.bool, device=to_device)
IO_tensors = {
    "inputs" :
    {'input_tensor': input_tensor},
    "outputs" :{
        'out_labels': out_labels_tensor,
        'ct_feat': ct_feat_tensor, 
        'center_pos_embedding': center_pos_embedding_tensor,
        'out_scores': out_scores_tensor, 
        'out_orders': out_orders_tensor, 
        'out_masks': out_masks_tensor}
}

In [9]:
def run_trt_engine(context, engine, tensors):
    bindings = [None]*engine.num_bindings
    for idx, binding in enumerate(engine):
        tensor_name = engine.get_tensor_name(idx)
        if engine.get_tensor_mode(binding)==trt.TensorIOMode.INPUT:
            bindings[idx] = tensors['inputs'][tensor_name].data_ptr()
            if context.get_tensor_shape(tensor_name):
                context.set_input_shape(tensor_name, tensors['inputs'][tensor_name].shape)
        else:
            bindings[idx] = tensors['outputs'][tensor_name].data_ptr()
    context.execute_v2(bindings=bindings)

run_trt_engine(cf_context, cf_engine, IO_tensors)

In [10]:
print(out_scores[0][0][:10])
print(out_scores_tensor[0][0][:10])#tensorRT

tensor([0.8610, 0.8600, 0.8393, 0.8083, 0.8044, 0.7641, 0.7413, 0.7288, 0.7048,
        0.6847], device='cuda:0')
tensor([0.8610, 0.8600, 0.8393, 0.8083, 0.8044, 0.7641, 0.7413, 0.7288, 0.7048,
        0.6847], device='cuda:0')


In [11]:
print(ct_feat[0][0][:20])
print(ct_feat_tensor[0][0][:20])

tensor([0.0000, 0.0000, 0.4185, 0.2847, 0.0000, 0.0000, 0.0000, 0.0000, 0.2222,
        0.9349, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7531, 0.0000, 0.0000,
        0.0000, 0.8260], device='cuda:0')
tensor([0.0000, 0.0000, 0.4185, 0.2847, 0.0000, 0.0000, 0.0000, 0.0000, 0.2222,
        0.9349, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7531, 0.0000, 0.0000,
        0.0000, 0.8260], device='cuda:0')


In [12]:
print(out_orders[0][0][:20])
print(out_orders_tensor[0][0][:20])

tensor([49866, 62040, 59169, 57018, 61680, 98445, 59170, 50226, 30777, 57017,
        59529, 98446, 59530, 27541, 62041, 49867, 61679, 30776, 31137, 49506],
       device='cuda:0')
tensor([49866, 62040, 59169, 57018, 61680, 98445, 59170, 50226, 30777, 57017,
        59529, 98446, 59530, 27541, 62041, 49867, 61679, 30776, 31137, 49506],
       device='cuda:0', dtype=torch.int32)
